# {County, Muni, Year, Month} aggregation

- Group crashes by {County, Muni, Year, Month}
- Compute sums of {fatal,injury,property} x {victims,crashes}
- Save to Parquet and SQLite

In [1]:
from utz import *
from njdot.crashes import Crashes
from njdot.paths import CMYM_DB, CMYM_PQT
from nj_crashes.utils import sql

In [2]:
%%time
c = Crashes.load()
c

CPU times: user 24.9 s, sys: 12.5 s, total: 37.3 s
Wall time: 40.3 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24


In [3]:
%%time
dt = c.dt.dt
y = dt.year.rename('y')
m = dt.month.rename('m')
ym = (y.astype(str) + m.apply(lambda m: '%02d' % m)).rename('ym')
ym

CPU times: user 2.46 s, sys: 237 ms, total: 2.7 s
Wall time: 2.73 s


id
0          200112
1          200101
2          200104
3          200104
4          200104
            ...  
6077298    202105
6077299    202105
6077300    202106
6077301    202106
6077302    202106
Name: ym, Length: 6077303, dtype: object

In [4]:
idxc = ['cc', 'mc', y, m]
ycm = c.groupby(idxc)[['tk', 'ti', 'tv']].sum()
ycm

tk  ti  tv
cc mc y    m             
1  1  2001 1    0  14  36
           2    0   9  45
           3    0  23  44
           4    0  31  67
           5    0  11  34
...            ..  ..  ..
21 23 2021 8    0   2  11
           9    0   2  13
           10   0   8  23
           11   0   5  20
           12   0   2  20

[132768 rows x 3 columns]

In [5]:
cmym = (
    c
    .groupby([ *idxc, 'severity'])
    .size()
    .rename('num_crashes')
    .reset_index(level=4)
    .pivot(columns='severity', values='num_crashes')
    .fillna(0)
    .astype(int)
    .rename(columns={ s: f'{s}c' for s in 'fip' })
)
cmym = sxs(ycm, cmym)
cmym

tk  ti  tv  fc  ic  pc
cc mc y    m                         
1  1  2001 1    0  14  36   0  10  10
           2    0   9  45   0   6  21
           3    0  23  44   0   9  10
           4    0  31  67   0  19  14
           5    0  11  34   0   7  12
...            ..  ..  ..  ..  ..  ..
21 23 2021 8    0   2  11   0   2   5
           9    0   2  13   0   2   8
           10   0   8  23   0   6  11
           11   0   5  20   0   3  10
           12   0   2  20   0   2  11

[132768 rows x 6 columns]

In [6]:
cmym.to_parquet(CMYM_PQT)

In [7]:
%%time
cur = sql.write(
    cmym, 'cmym', CMYM_DB,
    idxs=[
        ('cc', 'mc', 'y', 'm')
    ],
    page_size=2**16,
)
cur

Removing /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/cmym.db
Writing 132768 rows to /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/cmym.db


CPU times: user 873 ms, sys: 105 ms, total: 978 ms
Wall time: 1.03 s


Wrote DB: 9465856 bytes
After indices: 11689984 bytes
After setting page_size=65536 and vacuum: 11468800 bytes


In [8]:
cur.execute("Select * from SQLite_master")
idxs = cur.fetchall()
idxs

[('table',
  'cmym',
  'cmym',
  2,
  'CREATE TABLE cmym (\n\tcc BIGINT, \n\tmc BIGINT, \n\ty INTEGER, \n\tm INTEGER, \n\ttk BIGINT, \n\tti BIGINT, \n\ttv BIGINT, \n\tfc BIGINT, \n\tic BIGINT, \n\tpc BIGINT\n)'),
 ('index', 'ix_cmym_mc', 'cmym', 3, 'CREATE INDEX ix_cmym_mc ON cmym (mc)'),
 ('index', 'ix_cmym_cc', 'cmym', 4, 'CREATE INDEX ix_cmym_cc ON cmym (cc)'),
 ('index', 'ix_cmym_y', 'cmym', 5, 'CREATE INDEX ix_cmym_y ON cmym (y)'),
 ('index', 'ix_cmym_m', 'cmym', 6, 'CREATE INDEX ix_cmym_m ON cmym (m)'),
 ('index',
  'cc_mc_y_m',
  'cmym',
  7,
  'CREATE INDEX cc_mc_y_m ON cmym(cc, mc, y, m)')]

In [9]:
import boto3
s3 = boto3.client('s3')

In [10]:
s3.upload_file(CMYM_DB, Bucket='nj-crashes', Key=f'njdot/data/{basename(CMYM_DB)}')